# E-Commerce Chatbot - Basic Implementation

This notebook implements a simple chatbot for handling product and order queries.

In [ ]:
!pip install langchain

In [13]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import requests
import nest_asyncio
import uvicorn
from fastapi import FastAPI
from langchain.memory import ConversationSummaryBufferMemory


In [ ]:
products_df = pd.read_csv('Data/Product_Information_Dataset.csv')
orders_df = pd.read_csv('Data/Order_Data_Dataset.csv')

products_df = products_df.fillna('')
orders_df = orders_df.fillna('')

print(f"Loaded {len(products_df)} products and {len(orders_df)} orders")
print("\nProduct columns:", products_df.columns.tolist())
print("\nOrder columns:", orders_df.columns.tolist())

In [14]:
class BaseLanguageModel:
    def __init__(self, model_name: str):
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModelForCausalLM.from_pretrained(model_name)
        
    def generate_response(self, prompt: str) -> str:
        inputs = self.tokenizer(prompt, return_tensors="pt")
        outputs = self.model.generate(**inputs, max_length=100)
        return self.tokenizer.decode(outputs[0], skip_special_tokens=True)

In [15]:
class EcommerceBot:
    def __init__(self, products_df, orders_df, model_name="mistralai/Mistral-7B-v0.1"):
        self.products_df = products_df
        self.orders_df = orders_df
        self.embedding_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
        self.language_model = BaseLanguageModel(model_name)
        self.memory = ConversationSummaryBufferMemory(
            max_token_limit=1000
        )

    def process_query(self, query, customer_id=None):
        # Get context from memory
        history = self.memory.load_memory_variables({})
        context = self._get_relevant_context(query, customer_id)
        
        # Create prompt with context
        prompt = f"""
        You are an e-commerce assistant helping customers with product and order inquiries.
        
        Context about the query:
        {context}

        Conversation history:
        {history.get('history', '')}

        Current query: {query}

        Please provide a helpful, natural response.
        """
        
        # Get response from language model
        response = self.language_model.generate_response(prompt)
        
        # Save to memory
        self.memory.save_context({"input": query}, {"output": response})
        return response

    def _get_relevant_context(self, query, customer_id):
        context = []
        
        # Get product context if needed
        if not self._is_order_query(query):
            product_info = self._get_product_context(query)
            if product_info:
                context.append("Product information:\n" + product_info)
        
        # Get order context if customer_id provided
        if customer_id:
            order_info = self._get_order_context(customer_id)
            if order_info:
                context.append("Order information:\n" + order_info)
        
        return "\n\n".join(context)

    def _get_product_context(self, query):
        product_texts = self.products_df['title'] + ' ' + self.products_df['description'].fillna('')
        product_embeddings = self.embedding_model.encode(product_texts.tolist())
        query_embedding = self.embedding_model.encode([query])
        
        similarities = cosine_similarity(query_embedding, product_embeddings)[0]
        top_indices = similarities.argsort()[-3:][::-1]
        
        products = []
        for idx in top_indices:
            product = self.products_df.iloc[idx]
            products.append(f"""
            Product: {product['title']}
            Description: {product['description']}
            Price: ${product['price']}
            Rating: {product['rating']} stars
            """)
        
        return "\n".join(products)

    def _get_order_context(self, customer_id):
        response = requests.get(f"http://127.0.0.1:8000/data/customer/{customer_id}")
        orders = response.json()
        
        if 'error' in orders:
            return None
            
        orders_info = []
        for order in orders[-3:]:  # Get last 3 orders
            orders_info.append(f"""
            Date: {order['Order_Date']}
            Product: {order['Product_Category']}
            Amount: ${order['Sales']}
            Shipping: ${order['Shipping_Cost']}
            Priority: {order['Order_Priority']}
            """)
        
        return "\n".join(orders_info)

    def _is_order_query(self, query):
        order_keywords = ['order', 'purchase', 'bought', 'delivery', 'shipping', 'status']
        return any(keyword in query.lower() for keyword in order_keywords)

In [17]:
bot = EcommerceBot(products_df, orders_df)



OSError: You are trying to access a gated repo.
Make sure to have access to it at https://huggingface.co/mistralai/Mistral-7B-v0.1.
401 Client Error. (Request ID: Root=1-675fc409-0f8f836f4ae4e9975a231fd5;bde93756-9311-4b75-815d-84e3e54e5d58)

Cannot access gated repo for url https://huggingface.co/mistralai/Mistral-7B-v0.1/resolve/main/config.json.
Access to model mistralai/Mistral-7B-v0.1 is restricted. You must have access to it and be authenticated to access it. Please log in.

In [ ]:
# Product query
print(bot.process_query("What are some good guitar strings for beginners?"))

# Order query
print(bot.process_query("What's my order status?", customer_id=37077))

# Follow-up query (uses conversation history)
print(bot.process_query("What about my other orders?"))